In [1]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np
import json

In [2]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

EMBEDDING_DIMENSIONALITY = 512

# Example list of documents
documents: list[str] = ["I just discovered the course. Can I join now?",]

# This will trigger the model download and initialization
embedding_model = TextEmbedding(model_name=model_handle,
                               dim=EMBEDDING_DIMENSIONALITY,)

embeddings = np.array(list(embedding_model.embed(documents)))
min(embeddings[0])

np.float64(-0.11726373885183883)

In [3]:
doc = 'Can I still join the course after the start date?'

q = list(embedding_model.embed(doc))

print(np.linalg.norm(q))

q[0].dot(q[0])

1.0


np.float64(1.0)

In [4]:
embeddings[0].dot(q[0])

np.float64(0.9008528895674548)

In [5]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]


str_documents = [json.dumps(d) for d in documents]

In [6]:
new_doc = np.array(list(embedding_model.embed(str_documents)))

for i, doc in enumerate(new_doc):
    print(f"Index {i}: {embeddings[0].dot(doc)}")

Index 0: 0.8124201160239617
Index 1: 0.8147200862693439
Index 2: 0.8114790508243966
Index 3: 0.7730366279823991
Index 4: 0.7879898901615203


In [7]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]

full_text_embeddings = np.array(list(embedding_model.embed(full_text)))

for i, doc in enumerate(full_text_embeddings):
    print(f"Index {i}: {embeddings[0].dot(doc)}")

Index 0: 0.851454319443226
Index 1: 0.8436594005975433
Index 2: 0.8408287224005012
Index 3: 0.7755157657626952
Index 4: 0.8086007917931166


In [8]:
TextEmbedding._get_model_description(model_name="BAAI/bge-small-en")

DenseModelDescription(model='BAAI/bge-small-en', sources=ModelSource(hf='Qdrant/bge-small-en', url='https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', _deprecated_tar_struct=True), model_file='model_optimized.onnx', description='Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', license='mit', size_in_GB=0.13, additional_files=[], dim=384, tasks={})

In [9]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
qd_client = QdrantClient("http://localhost:6333")

In [11]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"

In [12]:
collection_name = "zoomcamp-faq"

qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [13]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [15]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
question = 'I just discovered the course. Can I still join it?'

query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model_handle 
    ),
    limit=1,
    with_payload=True
)

query_points

QueryResponse(points=[ScoredPoint(id=14, version=2, score=0.88934696, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])